In [1]:
# Parsing ascii files from the eyelink into eye position and data table files
# Clara Kuper, September 2020

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import ipywidgets as widgets

In [4]:
f = open('../edf/ckts_NS.asc','r')
l = f.readlines()
read = False
line = 0

while not read:
    if l[line].find('BLOCK_START')>0:
        start_id = line
        read = True
    line += 1

raw_list = []
events = []
tr_nr = 0
mov_type = 'unknown' #unknown, saccade, blink, fixation
in_trial = False

for line in l[start_id:]:
    
    append_message = [tr_nr, mov_type]
    
    if line[0:8].isnumeric():
            position_data = np.hstack((line.split(),append_message))
            raw_list.append(position_data)
        
    else:    
        if line.find('SYNC_TIME')>=0:
            sync_time = 0
            continue
        if line.find('TRIAL_START')>=0:
            tr_nr += 1
            in_trial = True
            continue
        if line.find('TRIAL_END')>=0:
            in_trial = False
            continue
        if line.find('EXPERIMENT_END')>=0:
            break

        if in_trial:
            if line[0:3]=='MSG':
                # get time write to table
                if line.find('RESPONSE')>=0:
                    pass
                if line.find('GO_SIGNAL')>=0:
                    pass
                else:
                    print('other case')

            if line.find('SSACC')>=0:
                mov_type = 'saccade'
            if line.find('ESACC')>=0:
                mov_type = 'unknown'
            if line.find('SFIX')>=0:
                mov_type = 'fixation'
            if line.find('EFIX')>=0:
                mov_type = 'unknown' 
            if line.find('SBLINK')>=0:
                mov_type = 'blink'
            if line.find('EBLINK')>=0:
                mov_type = 'unknown'


FileNotFoundError: [Errno 2] No such file or directory: '../edf/ckts_NS.asc'

In [127]:
eye_position = pd.DataFrame(raw_list, columns = ['time', 'x_pos', 'y_pos', 'pupil', 'right', 'trial', 'move_type'])

In [128]:
no_x_pos_available = np.where(eye_position['x_pos'] == '.')
no_y_pos_available = np.where(eye_position['y_pos'] == '.')
no_pupil_available = np.where(eye_position['pupil'] == '.')

eye_position['x_pos'].iloc[no_x_pos_available] = '0'
eye_position['y_pos'].iloc[no_y_pos_available] = '0'
eye_position['pupil'].iloc[no_pupil_available] = '0'

eye_position['time'] = pd.to_numeric(eye_position['time'])
eye_position['x_pos'] = pd.to_numeric(eye_position['x_pos'])
eye_position['y_pos'] = pd.to_numeric(eye_position['y_pos'])
eye_position['pupil'] = pd.to_numeric(eye_position['y_pos'])
eye_position['trial'] = pd.to_numeric(eye_position['trial'])

array(['blink', 'fixation', 'saccade', 'unknown'], dtype=object)

In [136]:
# visualize eye position per trial
def plot_func(trial):
    
    trial_data = eye_position.iloc[np.where(eye_position['trial']== trial)[0]]
    trial_data['time'] = trial_data['time'] - trial_data['time'].iloc[0]  
    groups = trial_data.groupby('move_type')
    fig, axs = plt.subplots(1,1)
    for name, group in groups:
        axs.scatter(group['time'], group['y_pos'], label = name)
        axs.scatter(group['time'], group['x_pos'], label = name)
        axs.set_xlim([0,2500])
        axs.set_ylim([0,2000])
    axs.legend()


trial_slider = widgets.IntSlider(min=0,
    max=1115,                          
    step=1,
    description='Trial No:',
    value=1,
    continuous_update = False) #difference units drift

widgets.interact(plot_func, trial = trial_slider)

    

interactive(children=(IntSlider(value=1, continuous_update=False, description='Trial No:', max=1115), Output()…

<function __main__.plot_func(trial)>

2084

2298